In [3]:
import pandas as pd
import numpy as np
trips = pd.read_csv("psrc_trips.csv")

<ipython-input-3-5ab300b92f4a>:3: DtypeWarning: Columns (5,8,25,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,52,68,69,73,75,76,79,86,87,88,89,90,91,92,93,94,95,96,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,130,131,132,133,134,135,136,137,138,139,140,142,154) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("psrc_trips.csv")


In [39]:
trips = trips.dropna(subset=["o_tract10","d_tract10"])
trips = trips[(trips['o_tract10'].astype(str).str.startswith('53033')) & (trips['d_tract10'].astype(str).str.startswith('53033'))]
trips = trips[trips['o_tract10']!=trips['d_tract10']]
counts = trips.groupby(["o_tract10", "d_tract10"]).size().reset_index(name="count")
max_indices = counts.groupby("o_tract10")["count"].idxmax()
most_counted = counts.loc[max_indices]
most_counted

,o_tract10,d_tract10,count
5,5.303300e+10,5.303300e+10,112
66,5.303300e+10,5.303300e+10,18
114,5.303300e+10,5.303300e+10,8
131,5.303300e+10,5.303300e+10,51
221,5.303300e+10,5.303300e+10,35
...,...,...,...
15859,5.303303e+10,5.303303e+10,13
15870,5.303303e+10,5.303303e+10,4
15894,5.303303e+10,5.303303e+10,14
15916,5.303303e+10,5.303303e+10,5


In [40]:
most_counted.drop(columns="count").to_csv("second_toy.csv")
second_data = pd.read_csv("second_toy.csv", index_col=0)

In [9]:
import geopandas as gpd

# Load the shapefile
shapefile_path = 'kc_tract_10.shp'
gdf = gpd.read_file(shapefile_path)

# Check the CRS information
crs = gdf.crs
print("CRS information:", crs)

CRS information: PROJCS["NAD83(HARN) / Washington North (ftUS)",GEOGCS["NAD83(HARN)",DATUM["NAD83_High_Accuracy_Reference_Network",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6152"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",47],PARAMETER["central_meridian",-120.833333333333],PARAMETER["standard_parallel_1",47.5],PARAMETER["standard_parallel_2",48.7333333333333],PARAMETER["false_easting",1640416.66666667],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [20]:
coord = pd.read_csv("cross_centroid.csv", index_col=0)
proj_crs = gdf.crs
geo_crs = Proj(init='epsg:4326')
for idx, row in coord.iterrows():
  x = coord.iloc[idx,1]
  y = coord.iloc[idx,2]
  lon, lat = transform(proj_crs, geo_crs, x, y)
  coord.iloc[idx,1] = lon
  coord.iloc[idx,2] = lat
coord

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-20-f982aedd7215>:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(proj_crs, geo_crs, x, y)
<ipython-input-20-f982aedd7215>:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(proj_crs, geo_crs, x, y)
<ipython-input-20-f982aedd7215>:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-

,TRACT,centroid_x,centroid_y
0,53033022006,-122.199280,47.704347
1,53033032320,-122.124348,47.755007
2,53033031703,-122.149204,47.371884
3,53033031206,-122.184013,47.312184
4,53033032102,-121.967142,47.491168
...,...,...,...
393,53033000900,-122.270510,47.707497
394,53033022202,-122.214345,47.720697
395,53033032323,-122.111780,47.698610
396,53033021600,-122.258041,47.766761


In [25]:
coord.to_csv("coord_xy.csv")
coord[coord['TRACT']==53033010900]

,TRACT,centroid_x,centroid_y
136,53033010900,-122.319717,47.542204


In [38]:
class SecondModel():
  def __init__(self, toy, coord):
      self.toy = toy
      self.coord = coord

  def predict(self, origin):
    destination  = self.toy[self.toy['o_tract10']==origin]['d_tract10'].values[0]
    o_coord = self.coord[self.coord['TRACT']==origin][["centroid_x", "centroid_y"]].values
    d_coord = self.coord[self.coord['TRACT']==destination][["centroid_x", "centroid_y"]].values
    return o_coord, d_coord, destination

toy = pd.read_csv("second_toy.csv", index_col=0)
coord = pd.read_csv("coord_xy.csv", index_col=0)
model = SecondModel(toy, coord)
model.predict(53033010100)[2]

53033010300.0